# Import posts from the old blog into this one

This should have already been run, but leaving it here in case we need to re-run again

In [1]:
from glob import glob
from yaml import safe_load
import nbformat as nbf
from pathlib import Path

In [28]:
path_posts = Path("../choldgraf.github.io/content/")
path_new_posts = Path("./posts")

title = None
for post in list(path_posts.rglob("**/*.md")) + list(path_posts.rglob("**/*.ipynb")):
    if ".ipynb_checkpoints" in str(post):
        continue
    path_rel_root = post.relative_to(path_posts)
    text = post.read_text()
    newlines = []
    # Clean up the text metadata
    lines = text.split('\n')
    for line in lines:
        # Tags should have no square brackets
        if line.strip().startswith("tags:"):
            line = line.replace('[', '').replace(']', '')
            
        # Replace explicit title with markdown header
        if line.strip().startswith("title:"):
            title = line.strip().split("title:")[-1].strip()            
        
        # Redirect for old links
        line = line.replace("permalink:", "redirect:")

        # Append the new line
        newlines.append(line)
    
    path_new_posts.joinpath(path_rel_root)
    
    # Write the new file
    path_new_post = path_new_posts.joinpath(path_rel_root)
    path_new_post.parent.mkdir(parents=True, exist_ok=True)
    path_new_post.write_text("\n".join(newlines))            
    
    # If it's a notebook, read it in and move the metadata to notebook metadata
    if post.suffix == ".ipynb":
        ntbk = nbf.reads(path_new_post.read_text(), nbf.NO_CONVERT)
        if ntbk.cells[0]['source'].startswith("---"):
            metadata = [ii for ii in ntbk.cells.pop(0)['source'].split('\n') if '---' not in ii]
            metadata = safe_load('\n'.join(metadata))
            if metadata.get("date"):
                metadata['date'] = str(metadata.get('date'))
            ntbk.metadata.update(metadata)
            with path_new_post.open("w") as ff:
                nbf.write(ntbk, ff)